In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

# ee.Initialize(project="ee-india-reservoirs")
ee.Initialize(project="ee-sahellakes")
# ee.Initialize(project="thurgau-irrigation")


In [3]:
import importlib
from src.et_green.compute_et_green import compute_et_green, calculate_band_std_dev
from src.et_green.filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)
from src.et_green import exporting_utils
importlib.reload(exporting_utils)
from src.et_green.exporting_utils import (
    process_et_green, 
    prepare_rainfed_fields,
    process_et_green_RF,
    process_et_green_RF_uncertainty
    )

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
    is_image_empty,
    fill_gaps_with_zeros
)

from utils.date_utils import print_collection_dates, merge_same_date_images

---

## Constants

In [4]:
YEAR = 2018

# KANTON = "FribourgAndVaud"
# KANTON = "Thurgau"

# PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"
# PATH_TO_AOI = f"projects/thurgau-irrigation/assets/{KANTON}/Bezirke_Contours_simplified_WGS84_Stammheim"
# PATH_TO_AOI = f"projects/thurgau-irrigation/assets/{KANTON}/Hoheitsgebiete_Contours_WGS84_simplified"
adm1_units = ee.FeatureCollection('projects/thurgau-irrigation/assets/GIS/Kantone_simplified100m')

#filter the cantons Zürich, Schaffhausen, and Thurgau from adm1_units using the 'NAME' property
adm1_units = adm1_units.filter(ee.Filter.inList('NAME', ['Zürich', 'Schaffhausen', 'Thurgau']))


TEMPORAL_RESOLUTION = "dekadal"

# # #Broye
# PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/{KANTON}/DoubleCropping/crop_vegetation_period_broye_{YEAR}"
# PATH_TO_JURISDICTION = (f"projects/thurgau-irrigation/assets/{KANTON}/elevation_bands_broye_v2_WGS84")
# PATH_TO_LANDUSE = (f"projects/thurgau-irrigation/assets/{KANTON}/filtered_permanent_pastures")
# LANDUSE_PROPERTY_NAME = "nutzung"

# Thurgau
PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/ZH_SH_TG/crop_vegetation_period_{YEAR}_harmonic"
# PATH_TO_JURISDICTION = PATH_TO_AOI#f"projects/thurgau-irrigation/assets/{KANTON}/Bezirke_Contours_simplified_WGS84_Stammheim"
PATH_TO_LANDUSE_2024 = f"projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_2024_Kulturen"
PATH_TO_LANDUSE = f"projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{YEAR}_Kulturen"
PATH_TO_RAINFED_REFERENCE_FIELDS = f"projects/thurgau-irrigation/assets/ZH_SH_TG/Rainfed_reference_fields/ZH_SH_TG_{YEAR}_Rainfed_b15"

LANDUSE_PROPERTY_NAME = "nutzung"
# PATH_TO_LANDUSE_ZH = "projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/ZH_Nutzungsflaechen_2022"
# LANDUSE_PROPERTY_NAME_ZH = "NUTZUNG"

rainfed_reference_set = {
    "Extensiv genutzte Weiden",
    "Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)",
    "Übrige Dauerwiesen (ohne Weiden)",
    "Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt",
    # "Übrige Grünfläche (Dauergrünflächen), nicht beitragsberechtigt",
    "Extensiv genutzte Wiesen (ohne Weiden)",
    "Weide (Heimweiden, üb. Weide ohne Sö.geb.)",
    "Weide (Heimweiden, üb. Weide ohne Sö.weiden)"
    # "Waldweiden (ohne bewaldete Fläche)",
    
    # "Kunstwiesen (ohne Weiden)",
}


ET_GREEN_BAND_NAME = "ET_green"

# #Landsat
ET_BAND_NAME = "ET"
SCALING_FACTOR = 100 ## relevant only for the ETF images, where DYNAMIC is False
DYNAMIC = True
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
NUMBER_OF_IMAGES = 21
EXPORT_IMAGE_RESOLUTION = 30  # in meters
ET_BAND_RESOLUTION = 30  # 30# in meters

KANTON = "Thurgau"
PATH_TO_ET_PRODUCT_1 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"
KANTON = "Zuerich"
PATH_TO_ET_PRODUCT_2 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"
KANTON = "Schaffhausen"
PATH_TO_ET_PRODUCT_3 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"

# ##Wapor
# ET_BAND_NAME ="downscaled"
# SCALING_FACTOR = 100  # scaling factor for the ET band to convert it back to float
# DYNAMIC = False #True #False, for wapor
# SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
# ET_BAND_RESOLUTION = 10  # 30# in meters
# NUMBER_OF_IMAGES = 21
# EXPORT_IMAGE_RESOLUTION = 10  # in meters
# # PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/WaPOR_10m_{YEAR}"
# PATH_TO_ET_PRODUCT = f"projects/thurgau-irrigation/assets/ET_products/{KANTON}/WaPOR_10m_{YEAR}"


MINIMUM_FIELD_SIZE = 10000  # in square meters
MAXIMUM_NUMBER_OF_TREES = 5
ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v3"
# ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_Gemeinden_{TEMPORAL_RESOLUTION}_from_WaPOR_10m"

print(ET_GREEN_ASSET_PATH)

PERIMETER_OF_INTEREST='projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew'


projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v3


## 1. Load Assets

In [5]:
# aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)
# aoi = ee.FeatureCollection(PATH_TO_AOI).geometry()
aoi = adm1_units.geometry()
jurisdictions = adm1_units

In [6]:
PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/ZH_SH_TG/crop_vegetation_period_{YEAR}_harmonic"

double_cropping_image = ee.Image(PATH_TO_DOUBLE_CROPPING_COLLECTION)
double_cropping_image = double_cropping_image.select('isDoubleCropping').where(double_cropping_image.select('secondStart').lte(5),0)
print('double cropping image:', double_cropping_image.bandNames().getInfo())

# Define vegetation period image for use in advanced processing
vegetation_period_image = ee.Image(PATH_TO_DOUBLE_CROPPING_COLLECTION)
print('veg period image:', vegetation_period_image.bandNames().getInfo())

double cropping image: ['isDoubleCropping']
veg period image: ['firstStart', 'firstEnd', 'secondStart', 'secondEnd', 'isDoubleCropping']


In [7]:

et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT_1)
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_2))
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_3))
    .filterDate(f"{YEAR}-05-01", f"{YEAR}-09-30")
    .map(
        lambda img: back_to_float(
            img, SCALING_FACTOR, DYNAMIC, SCALING_FACTOR_PROPERTY_NAME
        )
    )
)

# Mosaic images with identical dates
et_collection = merge_same_date_images(et_collection).sort("system:time_start")
# Sanity checks:
# print_collection_dates(et_collection)
et_collection_list = et_collection.toList(et_collection.size())

# print(f"Sizing of the ET collection: {et_collection.size().getInfo()}")

etf_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT_1 + "_ETF")
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_2 + "_ETF"))
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_3 + "_ETF"))
    .filterDate(f"{YEAR}-05-01", f"{YEAR}-09-30")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

# Mosaic images with identical dates
etf_collection = merge_same_date_images(etf_collection).sort("system:time_start")
print(f"Sizing of the ETF collection: {etf_collection.size().getInfo()}")
print_collection_dates(etf_collection)

etf_collection = etf_collection.toList(etf_collection.size())



Sizing of the ETF collection: 15
Dates of images in the collection:
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21


In [8]:
landuse_collection = ee.FeatureCollection(PATH_TO_LANDUSE)

landuse_collection = (
    landuse_collection.map(lambda f: f.set("nutzung", f.get(LANDUSE_PROPERTY_NAME)))
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else landuse_collection
)

print(
    f"Renamed {LANDUSE_PROPERTY_NAME} to 'nutzung'"
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else "Collection has 'nutzung' property"
)

rainfed_collection = ee.FeatureCollection(PATH_TO_RAINFED_REFERENCE_FIELDS) 

# #filter features with LANDUSE_PROPERTY_NAME equal to types present in rainfed_reference_set
# rainfed_collection = landuse_collection.filter(ee.Filter.inList(LANDUSE_PROPERTY_NAME, list(rainfed_reference_set)))
# #filter features with flaeche_m2 greater than 10000
# rainfed_collection = rainfed_collection.filter(ee.Filter.gt("flaeche_m2", 10000))

#check: aggregate_array of the 'nutzung' property
print("Rainfed reference crops in the collection:", rainfed_collection.aggregate_array('nutzung').distinct().getInfo())


n_trees= ee.FeatureCollection(PATH_TO_LANDUSE_2024).reduceToImage(properties=['anzahl_bae'], reducer=ee.Reducer.first())

Collection has 'nutzung' property
Rainfed reference crops in the collection: ['Übrige Dauerwiesen (ohne Weiden)']


In [9]:
forest_proximity = ee.Image('projects/thurgau-irrigation/assets/Wald_SWISSTLM3D_2023_proximity')  # Path to forest proximity image
rhiresD = ee.ImageCollection('projects/thurgau-irrigation/assets/Precipitation/RhiresD')  # Path to precipitation data
DEM = ee.ImageCollection('COPERNICUS/DEM/GLO30').select("DEM")  # Path to digital elevation model
proj = DEM.first().select(0).projection()
DEM = DEM.mosaic().setDefaultProjection(proj)

In [10]:
# PREPARE SOIL PROPERTY STACK
# ------------------ USER INPUTS ------------------
SOIL_FC_ASSET = 'projects/thurgau-irrigation/assets/Bodeneignungskarte_WGS84'  # <-- change

FIELDS = {
    'GRUNDIGKEI': 'gruend',   # rooting depth
    'SKELETT'    : 'skel',     # stone content
    'WASSERSPEI' : 'wsp',      # water storage capacity
    'NAHRSTOFF'  : 'nsp',      # nutrient storage
    'WASSERDURC' : 'perm',     # permeability
    'VERNASS'    : 'wet',      # waterlogging
}

# Nominal codes that mean "known class"
# VALID_CODES = [1, 2, 3, 4, 5, 6]
UNKNOWN_CODES = [0, -9999]     # treat these as unknown
SRC_CODES = ee.List([1, 2, 3, 4, 5, 6])
DST_CODES = ee.List([0, 1, 2, 3, 4, 5])

# -------------------------------------------------

soil_fc = ee.FeatureCollection(SOIL_FC_ASSET)

def rasterize(fc, prop):
    """Rasterize one attribute (no masking of unknowns)."""
    img = fc.reduceToImage(properties=[prop], reducer=ee.Reducer.first())
    return img.toInt16()

def get_present_codes_via_stratified(code_img, region, scale, unknown_codes):
    """
    Get the list of distinct class codes present in 'region'
    using stratifiedSample (1 sample per class is enough).
    Returns a client-side Python list of ints.
    """
    # Important: classBand must be integer
    img = ee.Image(code_img).toInt16().rename('class')

    # numPoints=1 will try to take one sample per class present
    fc = img.stratifiedSample(
        numPoints=1,
        region=region,
        scale=scale,
        geometries=False,
        dropNulls=True,
    )

    codes = fc.aggregate_array('class')
    # Remove unknown codes
    codes = codes.removeAll(ee.List(unknown_codes))
    return codes.getInfo()  # Convert to client-side Python list

def ordinal_with_unknown(code_img, short):
    """
    Returns two bands:
      short_ord: Int8 in {0..5, -1}  (ordered; -1 = unknown)
      short_UNK: 0/1 (1 where unknown inside polygons)
    Outside polygons remain masked.
    """
    shortS = ee.String(short)

    # Ordinal remap: 1..6 -> 0..5 ; everything else -> -1
    ord_img = code_img.remap(SRC_CODES, DST_CODES, -1).toInt8().rename(shortS.cat('_ord'))

    # Inside polygons (boolean)
    inside = code_img.mask().gt(0)

    # Unknown wherever ord_img == -1 AND inside polygons
    unk = ord_img.eq(-1).And(inside).toInt8().rename(shortS.cat('_UNK'))

    return ord_img.addBands(unk)

def one_hot_dynamic_with_unknown(code_img, valid_codes, prefix):
    """
    One-hot for each valid code + an _UNK band (0/1).
    Outside polygons remain masked.
    """
    # valid_codes is already a client-side Python list
    
    # Create one-hot encoded bands for valid codes using Python list comprehension
    dummies = []
    for c in valid_codes:
        band_name = f'{prefix}_{c}'
        dummy_band = code_img.eq(ee.Number(c)).rename(band_name).toInt8()
        dummies.append(dummy_band)
    
    # Combine dummy bands
    if dummies:
        dummy_image = ee.Image.cat(dummies)
    else:
        # If no valid codes, create an empty image
        dummy_image = ee.Image().select([])

    # Determine known areas (any valid code)
    if valid_codes:
        known = code_img.eq(ee.Number(valid_codes[0]))
        for c in valid_codes[1:]:
            known = known.Or(code_img.eq(ee.Number(c)))
    else:
        known = ee.Image.constant(0).rename('known')

    # Determine inside polygons as boolean
    inside = code_img.mask().gt(0)

    # Determine unknown areas (inside & not known)
    unknown = inside.And(known.Not()).rename(f'{prefix}_UNK').toInt8()

    # Combine all bands into a single image
    if dummies:
        return dummy_image.addBands(unknown)
    else:
        return unknown

# Build predictor stack
stacks = []
for field_name, short in FIELDS.items():
    band = rasterize(soil_fc, field_name)

    stacks.append(ordinal_with_unknown(band, short))

    ## this is not needed (use only with truly nominal data)
    # # Gather present codes for THIS property within AOI
    # present_codes = get_present_codes_via_stratified(
    #     code_img=band, region=aoi, scale=30, unknown_codes=UNKNOWN_CODES
    # )
    # print('Present codes for', field_name, ':', present_codes)
    
    # One-hot encode using the per-field present codes + UNK
    # stacks.append(one_hot_dynamic_with_unknown(band, present_codes, short))


soil_properties = ee.Image.cat(stacks)

print('Predictor bands:', soil_properties.bandNames().getInfo())

geometry = ee.Geometry.Point([8.820216405081002, 47.68266662551714])

# Sample value using reduceRegion
reduced = soil_properties.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=geometry,
    scale=30
)
print('Reduced values:', reduced.getInfo())

# Sample your predictor image at the point
sample_fc = soil_properties.reproject(
    crs='EPSG:4326',
    scale=30
    ).sample(region=geometry, scale=30)

# Get the first (and only) sampled feature
sample_feat = sample_fc.first()

# Get its dictionary of band values
props = sample_feat.toDictionary(soil_properties.bandNames())

print('Soil properties at point:', props.getInfo())


# alternative dataset: Bodenhinweiskarte
BODENHINWEIS_FC_ASSET = 'projects/thurgau-irrigation/assets/Bodenhinweiskarte'

depths = ['0_30_cm']               # add '60_120_cm' if useful
# (filenames like Soil_clay_0_30_cm.tif)
props = ['clay', 'silt', 'sand', 'soc', 'CECpot']  # skip pH for water storage

def load_soil(prop, depth):
    # Build asset ID
    img = ee.Image(f'{BODENHINWEIS_FC_ASSET}/Soil_{prop}_{depth}')
    img = img.toFloat()
    return img.rename(f'{prop}_{depth}')

# Load chosen layers
imgs = [load_soil(p, d) for p in props for d in depths]
soil_hi = ee.Image.cat(imgs)

# Optional: quick QA
print('Soil bands:', soil_hi.bandNames().getInfo())

# Sample your predictor image at the point
sample_fc = soil_hi.reproject(
    crs='EPSG:4326',
    scale=30
    ).sample(region=geometry, scale=30)

# Get the first (and only) sampled feature
sample_feat = sample_fc.first()

# Get its dictionary of band values
props = sample_feat.toDictionary(soil_hi.bandNames())
print('Soil properties at point:', props.getInfo())


Predictor bands: ['gruend_ord', 'gruend_UNK', 'skel_ord', 'skel_UNK', 'wsp_ord', 'wsp_UNK', 'nsp_ord', 'nsp_UNK', 'perm_ord', 'perm_UNK', 'wet_ord', 'wet_UNK']
Reduced values: {'gruend_UNK': 0, 'gruend_ord': 3, 'nsp_UNK': 0, 'nsp_ord': 4, 'perm_UNK': 0, 'perm_ord': 2, 'skel_UNK': 0, 'skel_ord': 1, 'wet_UNK': 0, 'wet_ord': 3, 'wsp_UNK': 0, 'wsp_ord': 3}
Soil properties at point: {'gruend_UNK': 0, 'gruend_ord': 3, 'nsp_UNK': 0, 'nsp_ord': 4, 'perm_UNK': 0, 'perm_ord': 2, 'skel_UNK': 0, 'skel_ord': 1, 'wet_UNK': 0, 'wet_ord': 3, 'wsp_UNK': 0, 'wsp_ord': 3}
Soil bands: ['clay_0_30_cm', 'silt_0_30_cm', 'sand_0_30_cm', 'soc_0_30_cm', 'CECpot_0_30_cm']
Soil properties at point: {'CECpot_0_30_cm': 139.6526336669922, 'clay_0_30_cm': 20.91111183166504, 'sand_0_30_cm': 47.90875244140625, 'silt_0_30_cm': 23.1572208404541, 'soc_0_30_cm': 1.9732916355133057}


## 2. Compute ET_green

In [11]:
# print(ET_GREEN_ASSET_PATH.replace("v3","v4"))
# ee.data.createAsset({'type': 'ImageCollection'}, ET_GREEN_ASSET_PATH.replace("v3","v4"),False)


In [ ]:
# You can customize these parameters for different runs:
nutzung_filter_list = [
    "Extensiv genutzte Weiden",
    "Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)",
    "Übrige Dauerwiesen (ohne Weiden)",
    "Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt",
    # "Übrige Grünfläche (Dauergrünflächen), nicht beitragsberechtigt",
    "Extensiv genutzte Wiesen (ohne Weiden)",
    "Weide (Heimweiden, üb. Weide ohne Sö.geb.)",
    "Weide (Heimweiden, üb. Weide ohne Sö.weiden)"
]
# Set to True to enable advanced processing (vegetation periods, extreme filtering, perimeter exclusion)
advanced_processing = False  

# ee.data.createAsset({'type': 'ImageCollection'}, ET_GREEN_ASSET_PATH.replace("v3","v4"))
# ET_GREEN_ASSET_PATH=ET_GREEN_ASSET_PATH.replace("v3","v4")

processed_et_collection = process_et_green_RF(
    et_collection_list=et_collection_list,
    rainfed_collection=rainfed_collection,
    year=YEAR,
    aoi=aoi,
    asset_path=ET_GREEN_ASSET_PATH,
    etf_collection_list=etf_collection,
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=ET_BAND_RESOLUTION,
    export_band_name="ET_green",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=10000
)

Filtered rainfed collection by nutzung values: ['Extensiv genutzte Weiden', 'Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)', 'Übrige Dauerwiesen (ohne Weiden)', 'Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt', 'Extensiv genutzte Wiesen (ohne Weiden)', 'Weide (Heimweiden, üb. Weide ohne Sö.geb.)', 'Weide (Heimweiden, üb. Weide ohne Sö.weiden)']
Sampled 0 points for training
Sampled 9989 points for training
Test ET_green sample: [290, 230, 250, 240, 270]
Exporting ET_green_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D2
Sampled 8747 points for training
Test ET_green sample: [382, 218, 227, 364, 300]
Exporting ET_green_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D3
Sampled 9906 points for training
Test ET_green sample: [280, 270, 280, 210, 280]
Exporting 

EEException: Computed value is too large.

In [11]:
##lets calculate the residuals as an uncertainty measure
nutzung_filter_list = [
    "Extensiv genutzte Weiden",
    "Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)",
    "Übrige Dauerwiesen (ohne Weiden)",
    "Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt",
    "Extensiv genutzte Wiesen (ohne Weiden)",
    "Weide (Heimweiden, üb. Weide ohne Sö.geb.)",
    "Weide (Heimweiden, üb. Weide ohne Sö.weiden)"
]

advanced_processing = False  

ETG_UNCERT_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Weiden_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
# ee.data.createAsset({'type': 'ImageCollection'}, ETG_UNCERT_ASSET_PATH)

processed_et_collection = process_et_green_RF_uncertainty(
    et_collection_list=et_collection_list,#.slice(11, 12),  # use only images from June to September
    rainfed_collection=rainfed_collection,
    year=YEAR,
    aoi=aoi,
    asset_path=ETG_UNCERT_ASSET_PATH,
    etf_collection_list=etf_collection,#.slice(11, 12),  # use only images from June to September
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=1000,
    export_band_name="ETgreen_residuals",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=20000
)

Filtered rainfed collection by nutzung values: ['Extensiv genutzte Weiden', 'Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)', 'Übrige Dauerwiesen (ohne Weiden)', 'Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt', 'Extensiv genutzte Wiesen (ohne Weiden)', 'Weide (Heimweiden, üb. Weide ohne Sö.geb.)', 'Weide (Heimweiden, üb. Weide ohne Sö.weiden)']
Total Sample size: 0
Total Sample size: 19757
Test residuals sample: [20, 20, 20, 60, 0]
Test ET_green sample: [210, 220, 210, 230, 230]
Test et_hat sample: [230, 240, 230, 290, 230]
Exporting ETgreen_residuals_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Weiden_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_05_D2
Total Sample size: 17208
Test residuals sample: [0, 46, 18, 36, 0]
Test ET_green sample: [218, 282, 209, 291, 282]
Test et_hat sample: [218, 236, 191, 255, 282]
Exporting ETgreen_residuals_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/asse

KeyboardInterrupt: 

In [26]:
# Process ET green for maize fields
# Enable advanced processing for nutzung categories that may include irrigated fields

# You can customize these parameters for different runs:
nutzung_filter_list = [
    "Silo- und Grünmais",
    "Körnermais"
]
# Set to True to enable advanced processing (vegetation periods, extreme filtering, perimeter exclusion)
advanced_processing = True  

ET_GREEN_ASSET_PATH=f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Mais_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
ee.data.createAsset({'type': 'ImageCollection'}, ET_GREEN_ASSET_PATH)

processed_et_collection = process_et_green_RF(
    et_collection_list=et_collection_list,
    rainfed_collection=landuse_collection.filter(ee.Filter.gt("flaeche_m2", 10000)),
    year=YEAR,
    aoi=aoi,
    asset_path=ET_GREEN_ASSET_PATH,
    etf_collection_list=etf_collection,
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=ET_BAND_RESOLUTION,
    export_band_name="ET_green",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=10000
)

Filtered rainfed collection by nutzung values: ['Silo- und Grünmais', 'Körnermais']
Excluding suspected irrigation regions from training: projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew
Added vegetation period bands to predictors
Sampled 0 points for training
Sampled 9595 points for training
Exporting ET_green_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Mais_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D2
Sampled 8401 points for training
Exporting ET_green_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Mais_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D3
Sampled 9470 points for training
Exporting ET_green_dekadal_2018_06_D1 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Mais_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_06_D1
Sampled 9591 points for training
Exporting ET_green_dekadal_2018_06_D2 for 2018 to projects/

In [27]:
##lets calculate the residuals as an uncertainty measure

##Field ID 00000000000000006468 has no mad_res value in 2019, why??
# First feature with no mad_res attribute: {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [0, 0]}, 'id': '00000000000000004104', 'properties': {'ETF_modeled': 0.8500000238418579, 'ET_blue': 12.686273668326583, 'ETa_ETc': 0.9672414062072628, 'bezugsjahr': 2021, 'canton': 'SH', 'class': 2, 'date': '2021-06-11', 'days': 10, 'decade': 2, 'field_id': '00000000000000006468', 'firstEnd': 5, 'firstStart': 4, 'flaeche_m2': 5649.762872009874, 'has_mad_res': 0, 'isDoubleCropping': 1, 'month': 6, 'n': 1, 'nutzung': 'Silo- und Grünmais', 'secondEnd': 9, 'secondStart': 6, 'unirrigated': 0, 'year': 2021, 'zero_fraction': 0.044962531223980015}}
nutzung_filter_list = [
    "Silo- und Grünmais",
    "Körnermais"
]
advanced_processing = True  

ETG_UNCERT_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Mais_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
ee.data.createAsset({'type': 'ImageCollection'}, ETG_UNCERT_ASSET_PATH)

processed_et_collection = process_et_green_RF_uncertainty(
    et_collection_list=et_collection_list,#.slice(11, 12),  # use only images from June to September
    rainfed_collection=landuse_collection.filter(ee.Filter.gt("flaeche_m2", 10000)),
    year=YEAR,
    aoi=aoi,
    asset_path=ETG_UNCERT_ASSET_PATH,
    etf_collection_list=etf_collection,#.slice(11, 12),  # use only images from June to September
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=1000,
    export_band_name="ETgreen_residuals",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=20000
)

Filtered rainfed collection by nutzung values: ['Silo- und Grünmais', 'Körnermais']
Excluding suspected irrigation regions from training: projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew
Added vegetation period bands to predictors
Total Sample size: 0
Total Sample size: 18663
Exporting ETgreen_residuals_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Mais_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_05_D2
Total Sample size: 16316
Exporting ETgreen_residuals_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Mais_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_05_D3
Total Sample size: 18538
Exporting ETgreen_residuals_dekadal_2018_06_D1 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Mais_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_06_D1
Total Sample size: 18659
Exporting ETgreen_residuals_dek

In [28]:
nutzung_filter_list = [
    "Kunstwiese (ohne Weiden)",
    "Kunstwiesen (ohne Weiden)"
]
rainfed_collection=landuse_collection.filter(ee.Filter.gt("flaeche_m2", 10000))

if nutzung_filter_list is not None:
    rainfed_collection = rainfed_collection.filter(ee.Filter.inList('nutzung', nutzung_filter_list))
    print(f"Filtered rainfed collection by nutzung values: {nutzung_filter_list}")
    
#Print the size of the filtered rainfed collection
print(f"Size of the filtered rainfed collection: {rainfed_collection.size().getInfo()}")


Filtered rainfed collection by nutzung values: ['Kunstwiese (ohne Weiden)', 'Kunstwiesen (ohne Weiden)']
Size of the filtered rainfed collection: 4535


In [29]:
# Process ET green for Kunstwiesen fields
# Enable advanced processing for nutzung categories that may include irrigated fields

# You can customize these parameters for different runs:
nutzung_filter_list = [
    "Kunstwiese (ohne Weiden)",
    "Kunstwiesen (ohne Weiden)"
]
# Set to True to enable advanced processing (vegetation periods, extreme filtering, perimeter exclusion)
advanced_processing = True  
ET_GREEN_ASSET_PATH=f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Kunstwiesen_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
ee.data.createAsset({'type': 'ImageCollection'}, ET_GREEN_ASSET_PATH)

processed_et_collection = process_et_green_RF(
    et_collection_list=et_collection_list,
    rainfed_collection=landuse_collection.filter(ee.Filter.gt("flaeche_m2", 10000)),
    year=YEAR,
    aoi=aoi,
    asset_path=ET_GREEN_ASSET_PATH,
    etf_collection_list=etf_collection,
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=ET_BAND_RESOLUTION,
    export_band_name="ET_green",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=10000
)

Filtered rainfed collection by nutzung values: ['Kunstwiese (ohne Weiden)', 'Kunstwiesen (ohne Weiden)']
Excluding suspected irrigation regions from training: projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew
Added vegetation period bands to predictors
Sampled 0 points for training
Sampled 9857 points for training
Exporting ET_green_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Kunstwiesen_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D2
Sampled 8803 points for training
Exporting ET_green_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Kunstwiesen_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D3
Sampled 9825 points for training
Exporting ET_green_dekadal_2018_06_D1 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Kunstwiesen_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_06_D1
Sampled 9857 points for training
Exporting ET_gree

In [30]:
##lets calculate the residuals as an uncertainty measure
nutzung_filter_list = [
    "Kunstwiese (ohne Weiden)",
    "Kunstwiesen (ohne Weiden)"
]
advanced_processing = True  

ETG_UNCERT_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Kunstwiesen_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
ee.data.createAsset({'type': 'ImageCollection'}, ETG_UNCERT_ASSET_PATH)

processed_et_collection = process_et_green_RF_uncertainty(
    et_collection_list=et_collection_list,#.slice(11, 12),  # use only images from June to September
    rainfed_collection=landuse_collection.filter(ee.Filter.gt("flaeche_m2", 10000)),
    year=YEAR,
    aoi=aoi,
    asset_path=ETG_UNCERT_ASSET_PATH,
    etf_collection_list=etf_collection,#slice(11, 12),  # use only images from June to September
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=1000,
    export_band_name="ETgreen_residuals",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=20000
)

Filtered rainfed collection by nutzung values: ['Kunstwiese (ohne Weiden)', 'Kunstwiesen (ohne Weiden)']
Excluding suspected irrigation regions from training: projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew
Added vegetation period bands to predictors
Total Sample size: 0
Total Sample size: 19338
Exporting ETgreen_residuals_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Kunstwiesen_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_05_D2
Total Sample size: 17019
Exporting ETgreen_residuals_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Kunstwiesen_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_05_D3
Total Sample size: 19167
Exporting ETgreen_residuals_dekadal_2018_06_D1 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Kunstwiesen_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_06_D1
Total Sample s

In [31]:
# Process ET green for Zuckerrüben fields
# Enable advanced processing for nutzung categories that may include irrigated fields

# You can customize these parameters for different runs:
nutzung_filter_list = [
    "Zuckerrüben"
]
# Set to True to enable advanced processing (vegetation periods, extreme filtering, perimeter exclusion)
advanced_processing = True  
ET_GREEN_ASSET_PATH=f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Zuckerrueben_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
ee.data.createAsset({'type': 'ImageCollection'}, ET_GREEN_ASSET_PATH)

processed_et_collection = process_et_green_RF(
    et_collection_list=et_collection_list,
    rainfed_collection=landuse_collection.filter(ee.Filter.gt("flaeche_m2", 10000)),
    year=YEAR,
    aoi=aoi,
    asset_path=ET_GREEN_ASSET_PATH,
    etf_collection_list=etf_collection,
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=ET_BAND_RESOLUTION,
    export_band_name="ET_green",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=10000
)

Filtered rainfed collection by nutzung values: ['Zuckerrüben']
Excluding suspected irrigation regions from training: projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew
Added vegetation period bands to predictors
Sampled 0 points for training
Sampled 7436 points for training
Exporting ET_green_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Zuckerrueben_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D2
Sampled 6548 points for training
Exporting ET_green_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Zuckerrueben_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D3
Sampled 7252 points for training
Exporting ET_green_dekadal_2018_06_D1 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Zuckerrueben_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_06_D1
Sampled 7409 points for training
Exporting ET_green_dekadal_2018_06_D2 for 2018 to projec

In [32]:
##lets calculate the residuals as an uncertainty measure
nutzung_filter_list = [
    "Zuckerrüben"
]
advanced_processing = True  

ETG_UNCERT_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Zuckerrueben_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
ee.data.createAsset({'type': 'ImageCollection'}, ETG_UNCERT_ASSET_PATH)

processed_et_collection = process_et_green_RF_uncertainty(
    et_collection_list=et_collection_list,#.slice(11, 12),  # use only images from June to September
    rainfed_collection=landuse_collection.filter(ee.Filter.gt("flaeche_m2", 10000)),
    year=YEAR,
    aoi=aoi,
    asset_path=ETG_UNCERT_ASSET_PATH,
    etf_collection_list=etf_collection,#.slice(11, 12),  # use only images from June to September
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name=ET_BAND_NAME,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=1000,
    export_band_name="ETgreen_residuals",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=20000
)

Filtered rainfed collection by nutzung values: ['Zuckerrüben']
Excluding suspected irrigation regions from training: projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew
Added vegetation period bands to predictors
Total Sample size: 0
Total Sample size: 13748
Exporting ETgreen_residuals_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Zuckerrueben_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_05_D2
Total Sample size: 12245
Exporting ETgreen_residuals_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Zuckerrueben_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_05_D3
Total Sample size: 13220
Exporting ETgreen_residuals_dekadal_2018_06_D1 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Zuckerrueben_dekadal_from_Landsat_30m_v4/ETgreen_residuals_dekadal_2018_06_D1
Total Sample size: 13522
Exporting ETgreen_residuals_

## 3. Compute ETF

In [11]:
# You can customize these parameters for different runs:
nutzung_filter_list = [
    "Extensiv genutzte Weiden",
    "Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)",
    "Übrige Dauerwiesen (ohne Weiden)",
    "Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt",
    # "Übrige Grünfläche (Dauergrünflächen), nicht beitragsberechtigt",
    "Extensiv genutzte Wiesen (ohne Weiden)",
    "Weide (Heimweiden, üb. Weide ohne Sö.geb.)",
    "Weide (Heimweiden, üb. Weide ohne Sö.weiden)"
]
# Set to True to enable advanced processing (vegetation periods, extreme filtering, perimeter exclusion)
advanced_processing = False  

ETF_ASSET_PATH = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ETF/ETF_Weiden_{TEMPORAL_RESOLUTION}_from_Landsat_30m_v4"
# ee.data.createAsset({'type': 'ImageCollection'}, ETF_ASSET_PATH)

processed_et_collection = process_et_green_RF(
    et_collection_list=etf_collection,
    rainfed_collection=rainfed_collection,
    year=YEAR,
    aoi=aoi,
    asset_path=ETF_ASSET_PATH,
    etf_collection_list=etf_collection,
    forest_proximity=forest_proximity,
    rhiresD=rhiresD,
    DEM=DEM,
    soil_properties=soil_hi,#soil_properties,
    n_trees=n_trees,
    et_band_name='ETF',
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=ET_BAND_RESOLUTION,
    export_band_name="ETF",
    max_trees=MAXIMUM_NUMBER_OF_TREES,
    nutzung_filter_list=nutzung_filter_list,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST,
    numPixels=10000
)

Filtered rainfed collection by nutzung values: ['Extensiv genutzte Weiden', 'Weiden (Heimweiden, übrige Weiden ohne Sömmerungsweiden)', 'Übrige Dauerwiesen (ohne Weiden)', 'Übrige Grünfläche (Dauergrünfläche), beitragsberechtigt', 'Extensiv genutzte Wiesen (ohne Weiden)', 'Weide (Heimweiden, üb. Weide ohne Sö.geb.)', 'Weide (Heimweiden, üb. Weide ohne Sö.weiden)']
Sampled 0 points for training
Sampled 9989 points for training
Exporting ETF_dekadal_2018_05_D2 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ETF/ETF_Weiden_dekadal_from_Landsat_30m_v4/ETF_dekadal_2018_05_D2
Sampled 8747 points for training
Exporting ETF_dekadal_2018_05_D3 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ETF/ETF_Weiden_dekadal_from_Landsat_30m_v4/ETF_dekadal_2018_05_D3
Sampled 9906 points for training
Exporting ETF_dekadal_2018_06_D1 for 2018 to projects/thurgau-irrigation/assets/ZH_SH_TG/ETF/ETF_Weiden_dekadal_from_Landsat_30m_v4/ETF_dekadal_2018_06_D1
Sampled 9989 points for training
Export

### New Parameters for process_et_green_RF

The `process_et_green_RF` function now supports enhanced parameters:

1. **`nutzung_filter_list`**: A list of 'nutzung' values to filter the rainfed_collection. 
   - If `None`, all features in the rainfed_collection are used
   - If provided, only features with 'nutzung' values in this list will be used

2. **`vegetation_period_image`**: An Earth Engine Image containing vegetation period information
   - Should have bands like 'firstStart', 'firstEnd', 'secondStart', 'secondEnd', 'isDoubleCropping'
   - This is typically the output from vegetation period extraction

3. **`advanced_processing`**: Boolean flag to enable comprehensive advanced processing for nutzung categories that may include irrigated fields
   - `False` (default): basic processing without advanced features
   - `True`: enables multiple advanced features:
     - **Vegetation period bands** are added as additional predictors for the classifier
     - **Extreme filtering**: removes ET values below 2nd and above 98th percentiles to reduce noise
     - **Perimeter exclusion**: excludes suspected irrigation regions from training data

4. **`perimeter_of_interest`**: Asset path to suspected irrigation regions to exclude from training
   - Only used when `advanced_processing=True`
   - Helps ensure the model trains on truly rainfed areas

**Example usage:**
```python
# For crops that may include irrigated fields (e.g., maize)
nutzung_filter = ["Silo- und Grünmais", "Körnermais"]
advanced_processing = True  # Enable all advanced features

processed_et_collection = process_et_green_RF(
    # ... other parameters ...
    nutzung_filter_list=nutzung_filter,
    vegetation_period_image=vegetation_period_image,
    advanced_processing=advanced_processing,
    perimeter_of_interest=PERIMETER_OF_INTEREST
)
```

In [34]:
# # copy all assets from ET_green_Weiden_dekadal_from_Landsat_30m_v5 to ET_green_Weiden_dekadal_from_Landsat_30m
# assetdir = f"projects/thurgau-irrigation/assets/{'ZH_SH_TG/ET_green/ETg_residuals_Zuckerrueben_dekadal_from_Landsat_30m_v3'}"

# assets = ee.data.listAssets({"parent": assetdir})["assets"]
# for img in assets:
#     id = img['id']
#     # asset = ee.data.getAsset(id)
#     name = id.split('ZH_SH_TG/ET_green/ETg_residuals_Zuckerrueben_dekadal_from_Landsat_30m_v3/')[1]
#     # if name contains 2022, 2023, or 2024, skip
#     if '2022' in name or '2023' in name or '2024' in name:
#         print('skipping: ' + name)
#         continue

#     print('asset name: ' + id)
#     ee.data.deleteAsset(id)

In [15]:
# copy all assets from ET_green_Weiden_dekadal_from_Landsat_30m_v5 to ET_green_Weiden_dekadal_from_Landsat_30m
assetdir = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4"

assets = ee.data.listAssets({"parent": assetdir})["assets"]
for img in assets:
    id = img['id']
    # asset = ee.data.getAsset(id)
    name = id.split('ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/')[1]
    if '2018' in name :
        print('asset name: ' + id)
        ee.data.deleteAsset(id)
        # ee.data.copyAsset(id, 'projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ETg_residuals_Weiden_dekadal_from_Landsat_30m_v2/' + name)

asset name: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D2
asset name: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_05_D3
asset name: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_06_D1
asset name: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_06_D2
asset name: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_06_D3
asset name: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green/ET_green_Weiden_dekadal_from_Landsat_30m_v4/ET_green_dekadal_2018_07_D2


In [36]:
# # copy all assets from ET_green_Weiden_dekadal_from_Landsat_30m_v5 to ET_green_Weiden_dekadal_from_Landsat_30m
# assetdir = f"projects/ee-hydro4u/assets/{'snow_CentralAsia'}"

# assets = ee.data.listAssets({"parent": assetdir})["assets"]
# for img in assets:
#     id = img['id']
#     # asset = ee.data.getAsset(id)
#     name = id.split('snow_CentralAsia/')[1]
#     if 'glacier_intersection' in name :
#         print('asset name: ' + id)
#         ee.data.deleteAsset(id)

### Comparing WaPOR to Landsat

In [37]:
# wapor_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2019_dekadal_from_WaPOR_10m"
# ).map(lambda img: back_to_float(img, 100))

# landsat_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/FribourgAndVaud/ET_green/ET_green_2019_dekadal_from_Landsat_30m"
# ).map(lambda img: back_to_float(img, 100))